In [1]:
import os
import pandas as pd
from llama_cloud_services import LlamaExtract
from schemas import Enrollment2024_25  #This could be adjusted through schemas.py
from dotenv import load_dotenv

In [4]:
PDF_ROOT = "university_pdfs_test"
OUTPUT_ROOT = "output_1"
os.makedirs(OUTPUT_ROOT, exist_ok=True)  
AGENT_ID = "af9ebc4f-f95b-4257-8791-071bce5bccfe" #Different based on your LLamaCloud account
load_dotenv() #make sure the API key is in the .env file

True

In [6]:
extractor = LlamaExtract()

#uncomment the below line if you are creating the agent for the first time
# agent = extractor.create_agent(name = "enrollment-parser-2024", data_schema=Enrollment2024_25)

agent = extractor.get_agent(id = AGENT_ID)

#uncomment the following lines if you updated the schema
# agent.data_schema = Enrollment2024_25
# agent.save()
# agent = extractor.get_agent(id = AGENT_ID)


No project_id provided, fetching default project.


In [8]:
agent.data_schema

{'additionalProperties': False,
 'properties': {'Undergraduate_Headcount': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': "Total undergraduate headcount for the 2024–2025 academic year (Different than undergraduate FTE. Sometimes you need to combine both full-time and part time).Search around the tables to locate what type of enrollment information it is.Ex: Texas A&M's first table is just says 'total enrollment headcount', but if you look around the table, it's for combination of undergraduate and graduate, so it doesn't belong to this field.Only extract data for the 2024–2025 year or terms labeled Fall 2024, etcignore any data from other years or terms (e.g., 2023–2024, Fall 2023). it's possible for a school to have multiple campuses, so combine all campuses' count or online and in-person count if applicable. Do not derive or hallucinate the data unless the field is actually in the document."},
  'Undergraduate_Headcount_Full_Time': {'anyOf': [{'type': 'inte

The following two cell blocks extract all schools' info into one excel file per school.

In [31]:
def process_school(school_name, school_dir):
    combined   = {}
    first_keys = None

    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        outfile = os.path.join(OUTPUT_ROOT, f"{school_name}.xlsx")
        df.to_excel(outfile)
        print(f"Saved output to {outfile}")
    else:
        print(f"No PDF data found for {school_name}")

In [ ]:
# Loop over schools
for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue
    print(f"Processing school: {school}")
    process_school(school, school_dir)

print("Extraction complete.")

The following cell block extracts all the schools' info into one excel sheet but in different tabs.

In [11]:
OUTPUT_FILE = os.path.join(OUTPUT_ROOT, "all_schools.xlsx")

writer = pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl")

for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue

    combined   = {}
    first_keys = None
    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {school}/{fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        sheet_name = school[:31]
        df.to_excel(writer, sheet_name=sheet_name)
    else:
        print(f"No data for {school}.")

writer.close()
print(f"All schools written to {OUTPUT_FILE}")

Extracting data from ASU/P11817713-P11393130-P11833489.pdf


Extracting files: 100%|█████████████████████████████| 1/1 [01:11<00:00, 71.36s/it]


Extracting data from Bradley University/P21862068-P21425154-P21869099.pdf


Extracting files: 100%|█████████████████████████████| 1/1 [00:50<00:00, 50.71s/it]


Extracting data from California_state_university/P21878315-P21436983-P21882690.pdf


...

Extracting data from University_of_minesota/P11814086-P21430908-P11830336.pdf


Extracting files: 100%|█████████████████████████████| 1/1 [00:46<00:00, 46.02s/it]


Extracting data from stevens_institue_of_technology/P21870185-P21430712-P21875325.pdf


Extracting files: 100%|█████████████████████████████| 1/1 [00:36<00:00, 36.17s/it]

All schools written to output_1/all_schools.xlsx


In [13]:
#Combine all the tabs into one sheet if wanted
file_path   = "output_1/all_schools.xlsx"
output_path = "output_1/all_schools_combined.xlsx"

raw = pd.read_excel(file_path, sheet_name=None, index_col=0)

school_series = {
    school: df.iloc[:, 0]                      # first (only) value column
    for school, df in raw.items()
}

df_comb = pd.DataFrame(school_series).T
df_comb.index.name = "School"                 
df_comb.insert(0, "Year", "2024‑2025")

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    df_comb.to_excel(writer, sheet_name="Combined")

print("Saved:", output_path)

Saved: output_1/all_schools_combined.xlsx
